<a href="https://colab.research.google.com/github/anandraiyer/access_forums_eval/blob/main/testing_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
from itertools import combinations

In [2]:
data = pd.read_csv('/content/drive/MyDrive/final.csv')

In [3]:
data.columns

Index(['Thread', 'DateTime', 'Author', 'Post', 'ParentPosts', 'PostID',
       'ThreadID', 'AuthorID', 'OriginID', 'DialogAct', 'ParentID_List'],
      dtype='object')

In [4]:
df = data[['Thread','ThreadID','PostID','DateTime','Author','Post','ParentID_List']]

In [5]:
df.head()

,Thread,ThreadID,PostID,DateTime,Author,Post,ParentID_List
0,Testing the new Site!,1,0,2015-11-30 11:59:19.544597,Tim Ford,Thank you very much for all those who worked o...,-1
1,Netflix not accessible to blind people using a...,2,1,2015-11-30 12:05:11.019288,Tim Ford,"Hi All, For those out there who want to use N...",-1
2,Testing the new Site!,1,2,2015-11-30 12:06:01.976409,"Walker, Michael E","Hi Tim, the group is working fine. I got your ...",[0]
3,Testing the new Site!,1,3,2015-11-30 12:12:07.800324,ken lawrence,Should the JDH mail be deleted?,[0]
4,Netflix not accessible to blind people using a...,2,4,2015-11-30 12:14:27.873186,Greg Nickel,Will do…,[1]


In [6]:
from itertools import chain
from itertools import product
from itertools import starmap
from functools import partial
import networkx as nx

In [7]:
def get_conversation_dag(tid):
  temp = df[df['ThreadID']==tid]

  threads = list(temp['Thread'])
  postids = list(temp['PostID'])
  posts = list(temp['Post'])
  datetimes = list(temp['DateTime'])
  authors = list(temp['Author'])
  parentids = list(temp['ParentID_List'])

  G = nx.DiGraph()
  G.add_nodes_from(posts)

  edges = []
  for i in range(len(postids)):
    parent = parentids[i]
    child = postids[i]

    if parent == "-1":
      continue
    else:
      parent = int(parent[1:-1])
    edges.append((parent,child))
  G.add_edges_from(edges)
  return G

In [8]:
def get_subthreads(tid):
  temp = df[df['ThreadID']==tid]

  threads = list(temp['Thread'])
  postids = list(temp['PostID'])
  posts = list(temp['Post'])
  datetimes = list(temp['DateTime'])
  authors = list(temp['Author'])
  parentids = list(temp['ParentID_List'])

  G = nx.DiGraph()
  G.add_nodes_from(posts)

  edges = []
  for i in range(len(postids)):
    parent = parentids[i]
    child = postids[i]

    if parent == "-1":
      continue
    else:
      parent = int(parent[1:-1])
    edges.append((parent,child))
  G.add_edges_from(edges)
  chaini = chain.from_iterable
  roots = (n for n,d in G.in_degree() if d==0)
  leaves = (n for n,d in G.out_degree() if d==0)
  all_paths = partial(nx.all_simple_paths, G)
  ans = list(chaini(starmap(all_paths, product(roots, leaves))))
  return(ans)

In [9]:
def get_list_subthreads(cluster_name,threadID):
  conversations = []
  for i, val in enumerate(get_subthreads(threadID)):
    conversations.append(cluster_name+str(i)+":"+str(val)[1:-1].replace(',',''))
  return conversations

In [10]:
get_list_subthreads('C',4445)

['C0:21680 21681', 'C1:21680 21682', 'C2:21680 21683']

In [11]:
def read_clusters(filename):
    # Read provided data
    clusters = {}
    cfile = ""
    all_points = set()
    for line in filename:
        if ':' in line:
            cfile = ':'.join(line.split(':')[:-1]).split('/')[-1]
            line = line.split(":")[-1]
        cluster = {int(v) for v in line.split()}
        clusters.setdefault(cfile, []).append(cluster)
        for v in cluster:
            all_points.add("{}:{}".format(cfile, v))
    return clusters, all_points

In [12]:
def create_contingency_table(gold, auto):
  table = []
  names_gold = []
  names_auto = []
  for i, _ in gold.items():
    names_gold.append(i)
  for i, _ in auto.items():
    names_auto.append(i)
  for i, v1 in gold.items():
    table_row = []
    for j, v2 in auto.items():
      table_row.append(len(v1.intersection(v2)))
    table.append(table_row)
  table = np.array(table)
  sum_rows = np.sum(table, axis = 1)
  sum_cols = np.sum(table, axis = 0)
  return table, sum_rows, sum_cols, names_gold, names_auto

In [13]:
def get_n(gold):
  u = set([])
  for _,v in gold.items():
    u = u.union(set(v))
  return(len(u))

In [14]:
def get_length_clustering(s):
  list_len = []
  for _,v in s.items():
    list_len.append(len(v))
  return(list_len)

In [15]:
def get_points(gold):
  u = set([])
  for _,v in gold.items():
    u = u.union(set(v))
  return(u)

In [48]:
def get_vi(s1,s2):
  c,c_rows,c_cols,g_name, a_name = create_contingency_table(s1,s2)
  #print(c)
  N = get_n(s1)
  
  H_uv = 0.0
  I_uv = 0.0

  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)
  total = N
  for i in range(len(c_rows)):
    for j in range(len(c_cols)):
      if c[i][j] != 0:
        num = c[i][j]
        A = c[i][j] / X[i]
        B = c[i][j] / Y[j]
        H_uv = H_uv - (num / total) * math.log(num / total, 2)
        I_uv = I_uv + (num / total) * math.log(num * total / (X[i] * Y[j]), 2)
      else:
        continue
  max_score = math.log(total, 2)
  VI = H_uv - I_uv
    
  scaled_VI = VI / max_score
  #print(sum_vi,math.log(total,2),scaled_vi,1-scaled_vi)
  return round((1 - scaled_VI),3)

In [41]:
def get_one_to_one(s1,s2):
  contingency,rows_sums,col_sums,gold_name,auto_name = create_contingency_table(s1,s2)
  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)
  N = get_n(s1)
  B = nx.Graph()
  left_nodes = []
  for i in gold_name:
    left_nodes.append('Left_'+str(i))
  right_nodes = []
  for i in auto_name:
    right_nodes.append('Right_'+str(i))
  B.add_nodes_from(left_nodes, bipartite=0)
  B.add_nodes_from(right_nodes, bipartite=1)

  for i in range(len(X)):
    for j in range(len(Y)):
      B.add_edge(left_nodes[i], right_nodes[j], weight = contingency[i][j])
  
  matches = nx.algorithms.matching.max_weight_matching(B)
  one_one_ratio = 0.0
  for u,v in matches:
    one_one_ratio = one_one_ratio + (100.0 * B.get_edge_data(u,v)['weight']/N)
  one_one_ratio = one_one_ratio
  return(round(one_one_ratio,3))

In [42]:
def get_omega_score(s1,s2):
  n = get_n(s1)
  N = n*(n-1.0)/2.0
  all_points = get_points(s1)
  all_pairs = combinations(all_points,2)
  s1_count = {}
  s2_count = {}
  for i,j in all_pairs:
    for m,n in s1.items():
      if i in n and j in n:
        if (i,j) in s1_count:
          s1_count[(i,j)] = s1_count.get((i,j)) + 1
        else:
          s1_count[(i,j)] = 1
      else:
        if (i,j) in s1_count:
          s1_count[(i,j)] = s1_count.get((i,j)) + 0
        else:
          s1_count[(i,j)] = 0
    for m,n in s2.items():
      if i in n and j in n:
        if (i,j) in s2_count:
          s2_count[(i,j)] = s2_count.get((i,j)) + 1
        else:
          s2_count[(i,j)] = 1
      else:
        if (i,j) in s2_count:
          s2_count[(i,j)] = s2_count.get((i,j)) + 0
        else:
          s2_count[(i,j)] = 0
  #print(s1_count)
  #print(s2_count)
  count_s1_rev = {}
  count_s2_rev = {}
  for (i,j),k in s1_count.items():
    if count_s1_rev.get((k),-1)!=-1:
      count_s1_rev[k] = count_s1_rev.get(k).union(set([(i,j)]))
    else:
      count_s1_rev[k] = set([(i,j)])
  for (i,j),k in s2_count.items():
    if count_s2_rev.get((k),-1)!=-1:
      count_s2_rev[k] = count_s2_rev.get(k).union(set([(i,j)]))
    else:
      count_s2_rev[k] = set([(i,j)])
  #print(count_s1_rev)
  #print(count_s2_rev)
  i_max = max(list(count_s1_rev.keys()))
  j_max = max(list(count_s2_rev.keys()))
  min_ij = min(i_max,j_max)
  #print(min_ij)
  count_s1_s2_rev = {}
  for i in range(0,min_ij+1):
    inter = count_s1_rev.get(i,set([])).intersection(count_s2_rev.get(i,set([])))
    count_s1_s2_rev[i] = inter
  #print(count_s1_s2_rev)
  U_sum = 0.0
  for i in range(0,min_ij+1):
    U_sum = U_sum + len(count_s1_s2_rev.get(i,set([])))
  U_sum = U_sum / N
  E_sum = 0.0
  for i in range(0,min_ij+1):
    E_sum = E_sum + (len(count_s1_rev.get(i,set([])))*len(count_s2_rev.get(i,set([]))))
  E_sum = E_sum / (N * N)
  
  omega = (U_sum - E_sum)/(1.0- E_sum)
  return(round(omega,3))

In [43]:
def create_shen_precision_table(s1, s2):
  table = []

  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)

  a = 0
  b = 0
  for i, v1 in s1.items():
    table_row = []
    n_i = X[a]
    b = 0
    for j, v2 in s2.items():
      n_j = Y[b]
      table_row.append(len(v1.intersection(v2))/n_j)
      b = b+1
    table.append(table_row)
    a=a+1
  table = np.array(table)
  
  return table

In [44]:
def create_shen_recall_table(s1, s2):
  table = []
  
  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)

  a = 0
  b = 0
  for i, v1 in s1.items():
    table_row = []
    n_i = X[a]
    b = 0
    for j, v2 in s2.items():
      n_j = Y[b]
      table_row.append(len(v1.intersection(v2))/n_i)
      b = b+1
    table.append(table_row)
    a=a+1
  table = np.array(table)
  
  return table

In [45]:
def create_shen_F_table(s1, s2):
  prec_table = create_shen_precision_table(s1,s2)
  recall_table = create_shen_recall_table(s1,s2)
 
  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)

  a = 0
  b = 0
  max_f = []
  for i, v1 in s1.items():
    table_row = []
    n_i = X[a]
    b = 0
    for j, v2 in s2.items():
      n_j = Y[b]
      f = (2 * prec_table[a][b] * recall_table[a][b])/(prec_table[a][b] + recall_table[a][b])
      table_row.append(f)
      b = b+1
    max_f.append(max(table_row))
    a=a+1

  return max_f

In [46]:
def get_shen_f1(s1,s2):
  max_f_table = create_shen_F_table(s1,s2)
  sum_f = 0.0
  n = get_n(s1)
  X = get_length_clustering(s1)

  for i in range(len(max_f_table)):
    sum_f = sum_f + ( (X[i]/n) * max_f_table[i])
  return(round(sum_f,3))

In [ ]:
def get_wang_f1(s1,s2):

In [49]:
gold = {'C0':{1,2,3,4},'C1':{5,6}}
auto = {'C0':{1,2,5,6},'C1':{3,4}}
VI = get_vi(gold, auto)
one_one = get_one_to_one(gold,auto)
omega = get_omega_score(gold,auto)
shen_f1 = get_shen_f1(gold,auto)

print(VI, one_one, omega,shen_f1)

0.484 66.667 -0.071 0.667


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
